# DABstep Benchmark Qwen2.5-Coder-32B Baseline

This notebook will guide you though submitting a Qwen2.5-Coder-32B baseline to the DABstep leaderboard.

* Live 🤗 Leaderboard: https://huggingface.co/spaces/adyen/DABstep
* Benchmark 🤗 Dataset: https://huggingface.co/datasets/adyen/DABstep
* LLM Agent Framework by 🤗: https://github.com/huggingface/smolagents/tree/main



## Environment Setup

We need to setup:
* **HuggingFace Token:** In order to make free API calls to HuggingFace Inference API you must have a HF account, the API verifies this by checking your account's token. This token will not be used for anything else.
* **Benchmark context files:** In order to solve the benchmark tasks the agent will need to reference documentation and analyze data which is spread out across multiple files, just like a real Data Analyst would.

### HuggingFace Token Setup

In [1]:
# %pip install smolagents==1.6.0 datasets==3.2.0

# # Try to install DABstep with build dependencies
# import subprocess
# import sys

# try:
#     # First ensure we have build tools
#     subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "setuptools", "wheel"],
#                          stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
#     # Then try to install DABstep
#     subprocess.check_call([sys.executable, "-m", "pip", "install", 
#                           "git+https://huggingface.co/spaces/adyen/DABstep.git@main"],
#                          stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
#     print("✓ DABstep package installed successfully")
# except Exception as e:
#     print(f"⚠️ Warning: DABstep package installation failed")
#     print("This is optional - the notebook will still work for basic agent functionality.")
#     print(f"Error details: {str(e)[:100]}")

In [3]:
import time
import os
import json
import datasets
import pandas as pd
from smolagents import CodeAgent
from smolagents.agents import ActionStep
from smolagents.models import HfApiModel
from huggingface_hub import login, hf_hub_download

# Try to import DABstep utilities, but allow the notebook to continue if unavailable
try:
    from dabstep_benchmark.utils import evaluate
    DABSTEP_AVAILABLE = True
except ImportError:
    print("Warning: DABstep benchmark utilities not available. Evaluation functions will not be available.")
    DABSTEP_AVAILABLE = False
    # Define a placeholder evaluate function
    def evaluate(*args, **kwargs):
        raise RuntimeError("DABstep utilities not installed. Please install DABstep package manually.")

try:
    login()
except Exception as e:
    print(f"Note: Login skipped or not required - {type(e).__name__}")

Note: Login skipped or not required - ImportError


#### Download context files
First we download the context files from the [Benchmark's Dataset](https://huggingface.co/datasets/adyen/DABstep) so that our agent can access them.


In [4]:
CONTEXT_FILENAMES = [
    "data/context/acquirer_countries.csv",
    "data/context/payments-readme.md",
    "data/context/payments.csv",
    "data/context/merchant_category_codes.csv",
    "data/context/fees.json",
    "data/context/merchant_data.json",
    "data/context/manual.md",
]
DATA_DIR = "/tmp/DABstep-data"
for filename in CONTEXT_FILENAMES:
    hf_hub_download(
        repo_id="adyen/DABstep",
        repo_type="dataset",
        filename=filename,
        local_dir=DATA_DIR,
        force_download=True
    )

CONTEXT_FILENAMES = [f"{DATA_DIR}/{filename}" for filename in CONTEXT_FILENAMES]

for file in CONTEXT_FILENAMES:
    if os.path.exists(file):
        print(f"{file} exists.")
    else:
        print(f"{file} does not exist.")

/tmp/DABstep-data/data/context/acquirer_countries.csv exists.
/tmp/DABstep-data/data/context/payments-readme.md exists.
/tmp/DABstep-data/data/context/payments.csv exists.
/tmp/DABstep-data/data/context/merchant_category_codes.csv exists.
/tmp/DABstep-data/data/context/fees.json exists.
/tmp/DABstep-data/data/context/merchant_data.json exists.
/tmp/DABstep-data/data/context/manual.md exists.


## Agent

Here we will setup a simple zero-shot prompt for the agent. It has two parts, the general prompt and then a quick outline of which files are available.

In [5]:
MODEL_ID = "Qwen/Qwen2.5-Coder-32B-Instruct"
HfApiModel(MODEL_ID)

## 3.4 Testing Agent

In [6]:
MODEL_ID = "Qwen/Qwen2.5-Coder-32B-Instruct"
MAX_STEPS = 7

agent = CodeAgent(
    tools=[],
    model=HfApiModel(MODEL_ID),
    additional_authorized_imports=["numpy", "pandas", "json", "csv", "os", "glob", "markdown"],
    max_steps=MAX_STEPS,
    verbosity_level=3,
)
# give agent power to open files
agent.python_executor.static_tools.update({"open": open})

In [6]:
PROMPT = """You are an expert data analyst and you will answer factoid questions by loading and referencing the files/documents listed below.
You have these files available:
{context_files}
Don't forget to reference any documentation in the data dir before answering a question.

Here is the question you need to answer:
{question}

Here are the guidelines you must follow when answering the question above:
{guidelines}
"""
question = "What are the unique set of merchants in the payments data?"
guidelines = "Answer with a comma separated list"

PROMPT = PROMPT.format(
    context_files=CONTEXT_FILENAMES,
    question=question,
    guidelines=guidelines
)

answer = agent.run(PROMPT)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an expert data analyst and you will answer factoid questions by loading and referencing the             │
│ files/documents listed below.                                                                                   │
│ You have these files available:                                                                                 │
│ ['/tmp/DABstep-data/data/context/acquirer_countries.csv', '/tmp/DABstep-data/data/context/payments-readme.md',  │
│ '/tmp/DABstep-data/data/context/payments.csv', '/tmp/DABstep-data/data/context/merchant_category_codes.csv',    │
│ '/tmp/DABstep-data/data/context/fees.json', '/tmp/DABstep-data/data/context/merchant_data.json',                │
│ '/tmp/DABstep-data/data/context/manual.md']                                                                     │
│ Don't forget to reference any documentation in the data dir before answering a question.                        │
│                                                                                                                 │
│ Here is the question you need to answer:                                                                        │
│ What are the unique set of merchants in the payments data?                                                      │
│                                                                                                                 │
│ Here are the guidelines you must follow when answering the question above:                                      │
│ Answer with a comma separated list                                                                              │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To answer this question, I'll load the payments data from the CSV file and extract the unique merchant    
names. I'll use pandas for handling the CSV data and then extract the unique values from the `merchant_name`       
column.                                                                                                            
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
import pandas as pd                                                                                                
                                                                                                                   
# Load payments data                                                                                               
payments_data = pd.read_csv('/tmp/DABstep-data/data/context/payments.csv')                                         
                                                                                                                   
# Extract unique merchant names                                                                                    
unique_merchants = payments_data['merchant_name'].unique()                                                         
                                                                                                                   
# Convert to comma separated list                                                                                  
unique_merchants_list = ",".join(unique_merchants)                                                                 
print(unique_merchants_list)                                                                                       
```                                                                                                                

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Load payments data                                                                                             
  payments_data = pd.read_csv('/tmp/DABstep-data/data/context/payments.csv')                                       
                                                                                                                   
  # Extract unique merchant names                                                                                  
  unique_merchants = payments_data['merchant_name'].unique()                                                       
                                                                                                                   
  # Convert to comma separated list                                                                                
  unique_merchants_list = ",".join(unique_merchants)                                                               
  print(unique_merchants_list)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'unique_merchants = payments_data['merchant_name'].unique()' due to: 
KeyError:'merchant_name'

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: Based on the inspection of the column names, the correct column to extract the unique set of merchants is 
`merchant`, not `merchant_name`. The observation shows that the `merchant` column exists in the `payments.csv`     
file, and the `Unique merchants` is currently empty because we didn't join the values yet. Let's extract the unique
values from the `merchant` column and convert them to a comma-separated list.                                      
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
import pandas as pd                                                                                                
                                                                                                                   
# Load payments data                                                                                               
payments_data = pd.read_csv('/tmp/DABstep-data/data/context/payments.csv')                                         
                                                                                                                   
# Extract unique merchant names                                                                                    
unique_merchants = payments_data['merchant'].unique()                                                              
                                                                                                                   
# Convert to comma separated list                                                                                  
unique_merchants_list = ",".join(unique_merchants)                                                                 
final_answer(unique_merchants_list)                                                                                
```                                                                                                                
                                                                                                                   

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Load payments data                                                                                             
  payments_data = pd.read_csv('/tmp/DABstep-data/data/context/payments.csv')                                       
                                                                                                                   
  # Extract unique merchant names                                                                                  
  unique_merchants = payments_data['merchant'].unique()                                                            
                                                                                                                   
  # Convert to comma separated list                                                                                
  unique_merchants_list = ",".join(unique_merchants)                                                               
  final_answer(unique_merchants_list)                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Crossfit_Hanna,Belles_cookbook_store,Golfclub_Baron_Friso,Martinis_Fine_Steakhouse,Rafa_AI

[Step 2: Duration 8.59 seconds| Input tokens: 7,816 | Output tokens: 548]

In [7]:
# You can inspect the steps taken by the agent by doing this
def clean_reasoning_trace(trace: list) -> list:
  for step in trace:
      # Remove memory from logs to make them more compact.
      if hasattr(step, "memory"):
          step.memory = None
      if isinstance(step, ActionStep):
          step.agent_memory = None
  return trace



for step in clean_reasoning_trace(agent.logs):
    print(step)

SystemPromptStep(system_prompt='You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important information you will then need.\nThese print outputs will then appear in the \'Observation:\' field, which will be available as input for the next step.\nIn the end you have to return a final answer using t

In [10]:
#lets explore data manually to understand what is going on here:

!ls -la



total 16
drwxr-xr-x 1 root root 4096 Nov  7 14:30 .
drwxr-xr-x 1 root root 4096 Nov 11 03:20 ..
drwxr-xr-x 4 root root 4096 Nov  7 14:30 .config
drwxr-xr-x 1 root root 4096 Nov  7 14:30 sample_data


## 3.5 Warmup Agent

We encourage to first get a good performing agent on this smaller dev split of the benchmark since running against the full benchmark can be very costly both in time and compute.

Once we are confident with our agent, we can try to create a submission to the leaderboard using the complete dataset of tasks


In [7]:
def run_benchmark(dataset: datasets.Dataset, agent: CodeAgent) -> list[dict]:
  agent_answers = []
  for task in dataset:
    tid = task['task_id']

    prompt = PROMPT.format(
      context_files=CONTEXT_FILENAMES,
      question=task['question'],
      guidelines=task['guidelines']
    )

    answer = agent.run(prompt)

    task_answer = {
        "task_id": str(tid),
        "agent_answer": str(answer),
        "reasoning_trace": str(clean_reasoning_trace(agent.logs))
    }

    agent_answers.append(task_answer)

  return agent_answers

In [8]:
MAX_STEPS = 7
MODEL_ID = "Qwen/Qwen2.5-Coder-32B-Instruct"

agent = CodeAgent(
    tools=[], # evaluating zero-shot capabilities of the model when writing code
    model=HfApiModel(MODEL_ID),
    additional_authorized_imports=["numpy", "pandas", "json", "csv", "os", "glob", "markdown"],
    max_steps=MAX_STEPS,
)

# give agent power to open files
agent.python_executor.static_tools.update({"open": open})

In [ ]:
PROMPT = """You are an expert data analyst and you will answer factoid questions by loading and referencing the files/documents listed below.
You have these files available:
{context_files}
Don't forget to reference any documentation in the data dir before answering a question.

Here is the question you need to answer:
{question}

Here are the guidelines you must follow when answering the question above:
{guidelines}
"""

SPLIT = "dev"
MAX_TASKS = 3 # It takes ~20 min to run the agent on all the dev set, so lets start just with the 2 first tasks
RUN_ID = int(time.time())


# load dataset from Hub
dev_task_dataset = datasets.load_dataset("adyen/DABstep", name="tasks", split=f"{SPLIT}[:{MAX_TASKS}]")
%time agent_answers = run_benchmark(dataset=dev_task_dataset, agent=agent)


README.md:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

all.jsonl:   0%|          | 0.00/189k [00:00<?, ?B/s]

dev.jsonl:   0%|          | 0.00/6.33k [00:00<?, ?B/s]

Generating default split:   0%|          | 0/450 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/10 [00:00<?, ? examples/s]

...

[Step 6: Duration 0.06 seconds| Input tokens: 18,726 | Output tokens: 1,390]

Reached max steps.

Final answer: Error in generating final LLM output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID:
Root=1-6841df00-63e8c1f97893c5653682ee8d;cb7d5876-ed4a-42a1-84b1-0cc48f3bc53e)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 7: Duration 0.06 seconds| Input tokens: 21,474 | Output tokens: 1,585]

CPU times: user 5.43 s, sys: 423 ms, total: 5.85 s
Wall time: 33.5 s


### Evaluation

In [ ]:
# Lets visualize the answers from our agent
pd.DataFrame(agent_answers)

In [ ]:
# Now we evaluate the answers
agent_answers = pd.DataFrame(agent_answers)
tasks_df = dev_task_dataset.to_pandas()
task_scores = evaluate(agent_answers=agent_answers, tasks_with_gt=tasks_df)

In [ ]:
# Inspect scores
task_scores = pd.DataFrame(task_scores)
task_scores["correct_answer"] = tasks_df["answer"]
task_scores["question"] = tasks_df["question"]
task_scores

## Final submission
Now we run the agent against the full benchmark and create a submission file to submit to the leaderboard


⚠️ *THIS WILL TAKE A LONG TIME IF YOU RUN THE FOR THE FULL BENCHMARK (450 tasks)* ⚠️

In [9]:
from pathlib import Path

# Crate dir to store agent run results
RUNS_DIR = Path().resolve() / "runs"
RUNS_DIR.mkdir(parents=True, exist_ok=True)

def write_jsonl(data: list[dict], filepath: Path) -> None:
    """Write a list of dictionaries to a JSONL file."""
    # Ensure the directory exists
    filepath.parent.mkdir(parents=True, exist_ok=True)

    with open(filepath, "w") as file:
        for entry in data:
            file.write(json.dumps(entry) + "\n")

In [ ]:
PROMPT = """You are an expert data analyst and you will answer factoid questions by loading and referencing the files/documents listed below.
You have these files available:
{context_files}

Here is the question you need to answer:
{question}

Here are the guidelines you must follow when answering the question above:
{guidelines}
"""

# now we run all dataset
SPLIT = "default"
RUN_ID = int(time.time())


# load dataset from Hub
benchmark_task_dataset = datasets.load_dataset("adyen/DABstep", name="tasks", split=SPLIT)
%time agent_answers = run_benchmark(dataset=benchmark_task_dataset, agent=agent)

write_jsonl(agent_answers, RUNS_DIR / f"{RUN_ID}.jsonl")

In [ ]:
# Lets visualize the answers from our agent
pd.DataFrame(agent_answers)

## Submission to Leaderboard

Your submission file should be saved in the `runs` folder of this directory.

Now is the fun part, were you submit it to the Leaderboard and see your score!

The leaderboard is here: https://huggingface.co/spaces/adyen/DABstep


### Analysis your submission scores

Once you have made a submission you can inspect the scores of your submission like these. Task scores are more granular task-level scores.

In [ ]:
submissions_dataset = datasets.load_dataset("adyen/DABstep", name="submissions", split="default")
task_scores_dataset = datasets.load_dataset("adyen/DABstep", name="task_scores", split="default")

AGENT_NAME = "Claude 3.5 Sonnet ReACT Baseline" # the agent name given in the submission form
ORGANISATION = "Adyen" # the organization name given in the submission form
SUBMISSION_ID = f"{ORGANISATION}-{AGENT_NAME}"

submissions_dataset_df = submissions_dataset.filter(lambda row: row["submission_id"] == SUBMISSION_ID).to_pandas()
task_scores_dataset_df = task_scores_dataset.filter(lambda row: row["submission_id"] == SUBMISSION_ID).to_pandas()

In [ ]:
submissions_dataset_df

In [ ]:
task_scores_dataset_df

## Next Steps

This is a few-shot ReAct prompt by deafult

Some things you can try next:

* do an error analysis and see why you failed certain questions
* tweaking the zero-shot prompt
* try a few-shot prompt
* try different agentic workflows like: CoT